In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
plt.rcParams['figure.dpi'] = 150
import seaborn as sns
import scipy.stats as st
import statsmodels
import statsmodels.api as sm
import vcf
from functools import reduce

import glob, os, yaml, subprocess, itertools, sparse, sys, statsmodels, shutil
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

#eigenvec_df = pd.read_csv("../PCA/eigenvec_100PC.csv", index_col=[0])
analysis_dir = '/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue'

lineages = pd.read_csv("../data/combined_lineages_samples.csv")
print(lineages.shape)
import collections, warnings
warnings.filterwarnings("ignore")

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "utils"))
from stats_utils import *
from data_utils import *

lineages_og = pd.read_csv("../data/lineages_Coll_Freschi.csv")
lineages_og['Sample Name'] = lineages_og['File_Name'].str.replace('-freebayes.vcf', '')

# coll2014 = pd.read_csv("../data/coll2014_SNP_scheme.tsv", sep="\t")
# coll2014["#lineage"] = coll2014["#lineage"].str.replace("lineage", "")
# coll2014.rename(columns={"#lineage": "Lineage"}, inplace=True)
# coll2014['nucleotide'] = [val.split('/')[1] for val in coll2014['allele_change'].values]

coll2014_full = pd.read_csv("Coll2014_SNPs_full.csv")

drug_abbr_dict = {"Delamanid": "DLM",
                  "Bedaquiline": "BDQ",
                  "Clofazimine": "CFZ",
                  "Ethionamide": "ETO",
                  "Linezolid": "LZD",
                  "Moxifloxacin": "MXF",
                  "Capreomycin": "CAP",
                  "Amikacin": "AMK",
                  "Pretomanid": "PMD",
                  "Pyrazinamide": "PZA",
                  "Kanamycin": "KAN",
                  "Levofloxacin": "LFX",
                  "Streptomycin": "STM",
                  "Ethambutol": "EMB",
                  "Isoniazid": "INH",
                  "Rifampicin": "RIF"
                 }

cc_df = pd.read_csv("../data/drug_CC.csv")
who_variants_V1 = pd.read_excel("../results/WHO-catalog-V1.xlsx", sheet_name='Mutation_catalogue')
who_variants = pd.read_csv("../results/WHO-catalog-V2.csv", header=[2]).query("tier==1").reset_index(drop=True)
del who_variants['mutation']

# results_final = pd.read_csv("../results/Regression_Final_Nov2023_Tier1.csv")
results_final = pd.read_csv("../results/Regression_Final_Jan2024_Tier1.csv")

drugs_lst = list(drug_abbr_dict.keys())

(52567, 7)


# Make File of Directories and File Names to Subset

In [2]:
lineages_for_F2 = lineages[['VCF_Dir', 'File_Name']]

# change directory because the files were moved
lineages_for_F2['VCF_Dir'] = [os.path.join("/n/scratch/users/s/sak0914/WHO_catalogue_data/VCF", os.path.basename(vcf_dir)) for vcf_dir in lineages_for_F2.VCF_Dir.values]

# remove file extension from the file names
lineages_for_F2['File_Name'] = ['.'.join(val.split('.')[:-1]) for val in lineages_for_F2['File_Name'].values]

In [4]:
fNames = []

for i, row in lineages_for_F2.iterrows():

    fName = os.path.join(row['VCF_Dir'], row['File_Name'] + '.vcf')
    
    if os.path.isfile(fName):
        fNames.append(fName)

len(fNames), len(lineages_for_F2)

(52567, 52567)

In [6]:
pd.Series(fNames).to_csv("VCF_fNames.csv", index=False, header=None)

In [ ]:
python3 -u lineages/compute_F2_score.py /n/scratch/users/s/sak0914/WHO_catalogue_data/VCF/6d441179-9b21-1f42-aa56-611440b8e6d6-03_ALSV06091971_NOR-43_LIB19752_1/03_ALSV06091971_NOR-43_LIB19752_1-freebayes_lineage_positions.vcf

In [305]:
lineages_for_F2['fName'] = lineages_for_F2['VCF_Dir'] + '/' + lineages_for_F2['File_Name'] + '_lineage_positions.vcf'

In [344]:
lineages_for_F2.fName.values[0]

'/n/scratch/users/s/sak0914/WHO_catalogue_data/VCF/11050333/11050333-freebayes_lineage_positions.vcf'

In [306]:
lineages_for_F2[['fName']].to_csv("VCF_fNames.tsv", index=False, header=None, sep='\t')

In [294]:
len(fNames_for_F2)

52567

In [4]:
print(lineages_for_F2.shape)
#lineages_for_F2.to_csv("../lineages/VCF_fNames.tsv", index=False, header=None, sep='\t')

(52567, 2)


In [ ]:
bcftools filter -i "POS == 1799921 | POS == 1816587" /n/scratch/users/s/sak0914/WHO_catalogue_data/VCF/SAMEA8718813/SAMEA8718813-freebayes.vcf

In [ ]:
bcftools filter -i "POS == 2026110 | POS == 3847221" /n/scratch/users/s/sak0914/WHO_catalogue_data/VCF/SAMEA8718813/SAMEA8718813-freebayes.vcf

In [206]:
#vcf_fName = '/n/scratch/users/s/sak0914/WHO_catalogue_data/VCF/6d441179-9b21-1f42-aa56-611440b8e6d6-03_7708-14_LIB9950_7708-14_LIB9950_1/03_7708-14_LIB9950_7708-14_LIB9950_1-freebayes.vcf'

vcf_fName = '/n/scratch/users/s/sak0914/WHO_catalogue_data/VCF/SAMN12838345/SAMN12838345-freebayes.vcf'

proc = subprocess.Popen(f"fast-lineage-caller {vcf_fName} --noheader --pass", shell=True, encoding='utf8', stdout=subprocess.PIPE)
output = proc.communicate()[0].replace('\n', '') # remove newline characters
output

'SAMN12838345-freebayes\tNA\tNA\tcanetti,westafrican1,ethiopian,tur,mungi\tNA\tNA'

In [274]:
# Lineage Defining SNPs - Import full set of Lineage Defining SNP sets from Coll et. al. 2014
lineage_defining_SNPs = pd.read_csv("./Coll2014_SNPs_full.csv").rename(columns={"#lineage": "lineage"})
lineage_defining_SNPs['lineage'] = [val.replace('lineage', '') for val in lineage_defining_SNPs['lineage'].values]

# According to Wyllie et. al. 2018, drop SNP sets corresponding to branches with fewer than 20 SNPs
# excluded_branches = ['1.2' , '3.1' , '3.1.2' , '4.1.2' , '4.3.4.2.1' , '4.6' , '4.7']
exclude_branch_SNP_thresh = 20

# use lineage as the counts column too, so the index and the column have the same name
excluded_branches = pd.DataFrame(lineage_defining_SNPs.groupby('lineage')['lineage'].count()).query("lineage < @exclude_branch_SNP_thresh").index.values
print(f"Excluding lineages {excluded_branches} with fewer than {exclude_branch_SNP_thresh} SNPs defining them")
full_length = len(lineage_defining_SNPs)

lineage_defining_SNPs = lineage_defining_SNPs.query("lineage not in @excluded_branches").reset_index(drop=True)
print(f"Keeping {len(lineage_defining_SNPs)}/{full_length} lineage-defining SNPs from the Coll 2014 scheme")

# separate REF and ALT alleles
lineage_defining_SNPs['REF'] = [val.split('/')[0] for val in lineage_defining_SNPs['allele_change'].values]
lineage_defining_SNPs['ALT'] = [val.split('/')[1] for val in lineage_defining_SNPs['allele_change'].values]

Excluding lineages ['1.2' '3.1' '3.1.2' '4.1.2' '4.3.4.2.1' '4.6' '4.7'] with fewer than 20 SNPs defining them
Keeping 6833/6915 lineage-defining SNPs from the Coll 2014 scheme


In [209]:
lineage_df = pd.read_csv("/n/scratch/users/s/sak0914/WHO_catalogue_data/VCF/11050333/11050333-freebayes_lineage_SNP_depths.csv")

In [340]:
def get_lineage_defining_SNP_depths(reduced_VCF):
    '''
    This function gets the minor alleles and depth information for all SNPs in the full Coll 2014 scheme. 
    '''
    
    lineage_df = pd.DataFrame(columns=['position', 'Depth', 'REF', 'ALT', 'Minor_Depth'])

    i = 0

    vcf_reader = vcf.Reader(filename=reduced_VCF)

    # iterate through each record from VCF file
    for record in vcf_reader:
    
        # check to see if call is at a lineage defining site AND that call is a SNP
        if ( record.POS in list( lineage_defining_SNPs.position ) ) and check_call_is_SNP(record.REF, record.ALT):
    
            # check that the SNP identities match too
            #if record.REF == lineage_defining_SNPs.query("position == @record.POS")['REF'].values[0] and record.ALT[0] == lineage_defining_SNPs.query("position == @record.POS")['ALT'].values[0]:
                
            ref_depth = record.INFO['RO']
            alt_depth = record.INFO['AO'][0]
            total_depth = record.INFO['DP']
    
            # check that the sum of the number of reads supporting ref and the number of reads supporting alt is ≤ total depth
            assert ref_depth + alt_depth <= total_depth
    
            # only need to consider a single alternative allele because the check_call_is_SNP already checks that there is a single alternative allele
            # find the allele with the lower depth (i.e., support)
            major_depth = np.max([ref_depth, alt_depth])
            minor_depth = total_depth - major_depth
            
            # minor_depth = np.min([ref_depth, alt_depth])
            # min_idx = [ref_depth, alt_depth].index(minor_depth)
            # minor_allele = list([record.REF, record.ALT[0]])[min_idx]
    
            # the minor depth should be less than or equal to 1/2 the full depth
            assert minor_depth <= total_depth / 2
            
            lineage_df.loc[i, :] = [record.POS, total_depth, record.REF, str(record.ALT[0]), minor_depth]
            i += 1

    # keep only variants that also match the REF and ALT identities in the Coll 2014 scheme. Merge right to keep all the SNPs
    # the SNPs that are NOT present have a minor depth of 0 because they are the REF allele only (no variants)
    lineage_df = lineage_df.merge(lineage_defining_SNPs[['position', 'lineage', 'REF', 'ALT']], on=['position', 'REF', 'ALT'], how='right')

    # fill NaNs in minor depth and 
    
    # check that the minor allele is one of REF and ALT
    # assert len(lineage_df.query("Minor_Allele != REF and Minor_Allele != ALT")) == 0
    return lineage_df

In [341]:
reduced_VCF = '/n/scratch/users/s/sak0914/WHO_catalogue_data/VCF/EUST15989/EUST15989-freebayes_lineage_positions.vcf'
lineage_df = get_lineage_defining_SNP_depths(reduced_VCF)

In [342]:
lineage_df.loc[pd.isnull(lineage_df['Minor_Depth'])].shape

(6635, 6)

In [343]:
lineage_df.loc[pd.isnull(lineage_df['Minor_Depth'])]

,position,Depth,REF,ALT,Minor_Depth,lineage
0,6112,NaN,G,C,NaN,1
1,8452,NaN,C,T,NaN,1
2,13298,NaN,G,C,NaN,1
3,22961,NaN,T,G,NaN,1
4,26347,NaN,C,G,NaN,1
...,...,...,...,...,...,...
6828,4292095,NaN,C,A,NaN,BOV_AFRI
6829,4330238,NaN,C,T,NaN,BOV_AFRI
6830,4386257,NaN,G,C,NaN,BOV_AFRI
6831,4393838,NaN,C,T,NaN,BOV_AFRI


In [318]:
# sum over m (minor depth), sum over d (total depth) for all SNPs in a lineage set
lineage_df_sets = lineage_df.groupby('lineage')[['Depth', 'Minor_Depth']].sum()

# minor allele fraction, p = M / D
lineage_df_sets['Minor_AF'] = lineage_df_sets['Minor_Depth'] / lineage_df_sets['Depth']

# get the two lineages with the largest minor allele frequencies
top_two_lineages = []

# iterate, and remove lineages that are already encompassed. i.e. if 2.2.1 is in the list already, then don't add 2 or 2.2
for query_lineage in lineage_df_sets.sort_values('Minor_AF', ascending=False).index.values:

    encompassed = False
    
    if len(top_two_lineages) == 2:
        break

    else:
        # if there are already lineages inside
        if len(top_two_lineages) > 0:
            for kept_lineage in top_two_lineages:

                # if the query lineage is a substring of a lineage that is already present, don't add it
                if query_lineage in kept_lineage:
                    encompassed = True

            if not encompassed:
                top_two_lineages.append(query_lineage)
        else:
            top_two_lineages.append(query_lineage)

print(f"Top two lineage sets: {np.sort(top_two_lineages)}")
F2_sets = lineage_df.query("lineage in @top_two_lineages")
F2_metric = F2_sets['Minor_Depth'].sum() / F2_sets['Depth'].sum()
F2_metric

Top two lineage sets: ['2.2.2' '4.6.2.1']


1.0

In [319]:
lineage_df_sets

,Depth,Minor_Depth,Minor_AF
lineage,,,
1.1.1.1,97,96,0.989691
2.2.1.2,191,190,0.994764
2.2.2,113,113,1.0
3,121,121,1.0
3.1.2.2,120,118,0.983333
4.1,9666,9620,0.995241
4.1.1.1,489,238,0.486708
4.1.2.1,173,173,1.0
4.2.2,119,39,0.327731


In [314]:
lineage_df

,position,Depth,REF,ALT,Minor_Depth,lineage
0,4013,111,T,C,111,4.9
1,11370,133,C,T,125,4.1
2,26959,130,C,G,129,4.9
3,30943,84,C,T,84,4.1
4,31077,117,C,T,117,4.1
...,...,...,...,...,...,...
193,4356110,128,G,C,126,4.9
194,4357123,119,C,A,119,4.1
195,4384417,107,C,T,107,4.1
196,4386709,145,C,T,145,4.1


In [259]:
lineages.query("Sample_Name=='11050333'")

,VCF_Dir,File_Name,Coll2014,Freschi2020,Sample_Name,Sample_ID,Lineage
0,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,11050333-freebayes.vcf,2.2.1,2.2.1.1.1.i3,11050333,810488,2


In [288]:
lineages.VCF_Dir.values[-1]

'/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue-raw-data/vcfs_freebayes/EUST15989'

In [249]:
lineage_df.query("lineage=='4.1.1.1'")

,position,Depth,REF,ALT,Minor_Allele,Minor_Depth,lineage
273,2586127,233,A,G,A,0,4.1.1.1
299,2866569,367,C,A,A,142,4.1.1.1
300,2866863,327,C,G,C,0,4.1.1.1


In [240]:
coll2014.query("Lineage=='4.1.1.1'")

,Lineage,position,allele_change,tag,nucleotide
23,4.1.1.1,1850119,C/T,coll2014,T


In [ ]:
bcftools filter -i "POS==2586127 | POS == 2866569 | POS == 2866863" /n/scratch/users/s/sak0914/WHO_catalogue_data/VCF/11050333/11050333-freebayes.vcf

In [237]:
92/389

0.2365038560411311

In [219]:
lineage_df_sets.sort_values('Minor_AF', ascending=False)

,Depth,Minor_Depth,Minor_AF
lineage,,,
4.2.2,389,92,0.236504
4.1.1.1,927,142,0.153182
2.2.1.2,753,51,0.067729
4.6.2.1,420,7,0.016667
6,211,3,0.014218
4.6.1,274,3,0.010949
4.4.1.2,274,3,0.010949
1.1.1.1,282,3,0.010638
4.4.1.1,291,3,0.010309


In [207]:
pd.DataFrame(dict(zip(['File_Name', 'Coll2014', 'Freschi2020', 'Lipworth2019', 'Shitikov2017', 'Stucki2016'], output.split('\t'))), index=[0])

,File_Name,Coll2014,Freschi2020,Lipworth2019,Shitikov2017,Stucki2016
0,SAMN12838345-freebayes,NA,NA,"canetti,westafrican1,ethiopian,tur,mungi",NA,NA


In [208]:
lineages

,VCF_Dir,File_Name,Coll2014,Freschi2020,Sample_Name,Sample_ID,Lineage
0,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,11050333-freebayes.vcf,2.2.1,2.2.1.1.1.i3,11050333,810488,2
1,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,SAMN21856363-freebayes.vcf,2.2.1,2.2.1.1.1.i3,SAMN21856363,34352,2
2,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,SAMEA1101763-freebayes.vcf,3,3.1.1,SAMEA1101763,57986,3
3,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,SAMEA1019058-freebayes.vcf,4.8,4.2.1.1.1.1.1.1.i1,SAMEA1019058,79833,4
4,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,SAMN08629568-freebayes.vcf,2.2.1,2.2.1.1.1,SAMN08629568,138326,2
...,...,...,...,...,...,...,...
52562,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,EUST16031-freebayes.vcf,4.1.2.1,4.1.i1.1.1.1,EUST16031,933863,4
52563,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,EUST07422-freebayes.vcf,2.2.1,2.2.1.1.1,EUST07422,933860,2
52564,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,EUST07438-freebayes.vcf,4.2.1,4.2.2.2.1,EUST07438,933861,4
52565,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,EUST07407-freebayes.vcf,2.2.1,2.2.1.1.1,EUST07407,933858,2


In [11]:
lineages.loc[pd.isnull(lineages['Coll2014'])]

,VCF_Dir,File_Name,Coll2014,Freschi2020,Sample_Name,Sample_ID,Lineage
9383,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,SAMN12838345-freebayes.vcf,NaN,NaN,SAMN12838345,119967,"canetti,westafrican1,ethiopian,tur,mungi"
9578,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,SPE_6B1800_D77B96EAA535-freebayes.vcf,NaN,NaN,SPE_6B1800_D77B96EAA535,166814,"canetti,westafrican1,ethiopian,tur,mungi"
15495,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,24_PT-674_2019-030_1-freebayes.vcf,NaN,NaN,24_PT-674_2019-030_1,171503,"canetti,westafrican1,ethiopian,stype,tur,mungi"
21145,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,SAMN07344637-freebayes.vcf,NaN,NaN,SAMN07344637,142816,"canetti,westafrican1,ethiopian,tur,mungi"
24374,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,24_PT-669_2018-579_1-freebayes.vcf,NaN,NaN,24_PT-669_2018-579_1,171499,"canetti,westafrican1,ethiopian,tur,mungi"
24441,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,SAMN12838344-freebayes.vcf,NaN,NaN,SAMN12838344,119968,"canetti,westafrican1,ethiopian,tur,mungi"
32720,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,SAMN07344639-freebayes.vcf,NaN,NaN,SAMN07344639,142814,"canetti,westafrican1,ethiopian,tur,mungi"
49050,/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-...,SAMN08437197-freebayes.vcf,NaN,NaN,SAMN08437197,96124,"canetti,westafrican1,ethiopian,tur,mungi"


In [127]:
lineage_df.Minor_Depth.sum() / lineage_df.Depth.sum()

0.3469656992084433

In [117]:
lineage_df.Minor_Depth.sum() / lineage_df.Depth.sum()

0.0008176614881439084

In [111]:
lineage_df.Minor_Depth.sum() / lineage_df.Depth.sum()

0.3475682087781732

In [88]:
record.INFO['PRO'], record.INFO['PAO']

(0.0, [0.0])

In [83]:
record.INFO['RO'] + record.INFO['AO'][0] == record.INFO['DP']

False

In [87]:
record.INFO['RO'], record.INFO['AO'], record.INFO['DP']

(38, [67], 106)

In [73]:
coll2014.query("position in @found_pos")

,Lineage,position,allele_change,tag,nucleotide
8,2,497491,G/A,coll2014,A
10,2.2,2505085,G/A,coll2014,A
14,2.2.2,346693,G/T,coll2014,T
20,4**,931123,T/C,coll2014,C
32,4.3,764995,C/G,coll2014,G
34,4.3.2,4316114,G/A,coll2014,A
35,4.3.2.1,3388166,C/G,coll2014,G
56,4.9**,1759252,G/T,coll2014,T


In [ ]:
bcftools filter -i "POS == 346693 | POS == 3388166"

In [57]:
vcf_reader = vcf.Reader(filename='/n/scratch/users/s/sak0914/WHO_catalogue_data/VCF/11050333/11050333-freebayes.vcf')

for record in vcf_reader:

    #print(record.REF, record.ALT, record.INFO['MQM'], record.QUAL, record.INFO['QA'], record.INFO['DP'], record.INFO['AN'], record.INFO['AC'])
    # print(record)
    for call in record.samples:
        print(call['GQ'])
        break

AttributeError: 'CallData' object has no attribute 'GQ'

In [58]:
call

Call(sample=11050333, CallData(GT=1/1, DP=171, AD=[0, 171], RO=0, QR=0, AO=171, QA=5812, GL=[-523.195, -51.4761, 0.0]))

In [61]:
gq = np.min(-10 * np.log(likelihood) for likelihood in call['GL'])
gq

<generator object <genexpr> at 0x7f8b90613510>

In [63]:
np.min([-10 * np.log(likelihood) for likelihood in call['GL']])

nan

In [46]:
record.FORMAT

'GT:DP:AD:RO:QR:AO:QA:GL'

In [49]:
record.FORMAT

'GT:DP:AD:RO:QR:AO:QA:GL'

In [45]:
record.ID

In [ ]:
				BQ = record.INFO['BQ']

				# Mean Mapping Quality @ locus
				MQ = record.INFO['MQM']

				# Number of Reads w/ Deletion 
				DC = record.INFO['DC']

				# Number of Reads w/ Insertion
				IC = record.INFO['IC']  
				
				# Depth of Valid Reads in Pileup
				VD = record.INFO['DP']

In [ ]:
FORMAT=<ID=GQ

In [39]:
descriptions = []

with open('/n/scratch/users/s/sak0914/WHO_catalogue_data/VCF/11050333/11050333-freebayes.vcf', "r") as file:
    for line in file.readlines():
        if line[0] == '#':
            descriptions.append(line)

In [41]:
for string in descriptions:
    if 'phred' in string.lower():
        print(string)

##INFO=<ID=QR,Number=1,Type=Integer,Description="Reference allele quality sum in phred">

##INFO=<ID=QA,Number=A,Type=Integer,Description="Alternate allele quality sum in phred">

##INFO=<ID=PQR,Number=1,Type=Float,Description="Reference allele quality sum in phred for partial observations">

##INFO=<ID=PQA,Number=A,Type=Float,Description="Alternate allele quality sum in phred for partial observations">

##INFO=<ID=SRP,Number=1,Type=Float,Description="Strand balance probability for the reference allele: Phred-scaled upper-bounds estimate of the probability of observing the deviation between SRF and SRR given E(SRF/SRR) ~ 0.5, derived using Hoeffding's inequality">

##INFO=<ID=SAP,Number=A,Type=Float,Description="Strand balance probability for the alternate allele: Phred-scaled upper-bounds estimate of the probability of observing the deviation between SAF and SAR given E(SAF/SAR) ~ 0.5, derived using Hoeffding's inequality">

##INFO=<ID=ABP,Number=A,Type=Float,Description="Allele balan